In [3]:
import os
import json
from collections import Counter, defaultdict

# Ruta base donde buscar los archivos .json
base_dir = "../datasets/fallos_json"

# Diccionario para almacenar la información
result = {}

# Recorrer todas las carpetas y archivos .json
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    citations = (
                        data.get("METADATOS", {})
                        .get("ARTICULOS_CITADOS", {})
                        .get("citations", [])
                    )
                    for citation in citations:
                        main_source = citation.get("main_source")
                        cited_articles = citation.get("cited_articles", [])
                        if main_source is not None and cited_articles is not None:
                            if main_source not in result:
                                result[main_source] = []
                            result[main_source].extend(cited_articles)
            except Exception as e:
                print(f"Error en {file_path}: {e}")

# Construir el JSON de salida
output = {}
for source, articles in result.items():
    freq = Counter(articles)
    output[source] = {
        "cited_articles_frequency": dict(freq),
        "total_cited_articles": sum(freq.values())
    }

# Guardar el resultado en un archivo
with open("main_source_cited_articles_stats.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("Archivo 'main_source_cited_articles_stats.json' generado correctamente.")

Archivo 'main_source_cited_articles_stats.json' generado correctamente.


# Prueba del procesador enriquecido de fallos legales

Vamos a procesar los archivos JSON usando el nuevo procesador y mostrar algunos fragmentos enriquecidos.

In [ ]:
from pathlib import Path
from backend.data.processing.enriched import EnrichedProcessor

# Ruta al directorio de fallos JSON
json_dir = Path("../../datasets/fallos_json/2024/07")

# Instanciar el procesador enriquecido
processor = EnrichedProcessor()

# Procesar el directorio y obtener los fragmentos enriquecidos
fragmentos = list(processor.process_directory(json_dir))

print(f"Total de fragmentos extraídos: {len(fragmentos)}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\MSI\AppData\Local\Temp\ipykernel_12552\783722374.py", line 2, in <module>
    from ..backend.data.processing.enriched import EnrichedProcessor
ImportError: attempted relative import with no known parent package

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
# Mostrar los primeros 3 fragmentos enriquecidos
for frag in fragmentos[:3]:
    print(f"Expte: {frag.expediente}")
    print(f"Sección: {frag.section}")
    print(f"Texto: {frag.text[:120]}...")
    print(f"Idea central: {frag.idea_central[:100] if frag.idea_central else None}")
    print(f"Artículos citados: {frag.articulos_citados}")
    print(f"Materia: {frag.materia_preliminar}")
    print("-" * 60)

Si ves los campos extra correctamente poblados, ¡el procesador está funcionando bien!